# CRC Workshop: Machine Learning with Functional Connectivity (FC) Data


## The AOMIC Data

First, lets load the data and inspect it a bit. The [AOMIC dataset](https://nilab-uva.github.io/AOMIC.github.io/) is a collection data obtained in three different studies (**PIOP1**, **PIOP2**, **ID1000**). Here, we will be only concerned with the data from the **ID1000** study, which aimed to collect 1000 fMRI scans during movie-watching. The next cell defines the path to all the **ID1000** specific data, and also adds the names of the two files we will be interested in. One of these files contains the preprocessed **functional connectivity (FC)** data, whereas the other file contains the important **demographic** information. Let's start by also loading the dependencies:

In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
# Path to ID1000 data within the AOMIC datalad dataset
ID1000_path = (
    Path("..") / "aomic-fc"/ "junifer_storage" / 
    "JUNIFER_AOMIC_TSV_CONNECTOMES" / "ID1000"
)

# Path to the demographics data file
demographics_path = ID1000_path / "ID1000_participants.tsv"

# Path to the connectomes data file
# The name of this file is a bit of a mouthful but contains important
# information
connectomes_path = ID1000_path / (
    "ID1000_BOLD_parccortical-Schaefer100x17FSLMNI_"
    "parcsubcortical-TianxS2x3TxMNInonlinear2009cAsym_"
    "marker-empiricalFC_moviewatching.tsv.gz"
)

### Demographic Data

Now that we have defined these paths let's load each file and look at them one by one. Let's start with the demographics. We will load it using pandas, and as you might see from the file extension, these are both **tsv** files and we will therefore load them using a tab as a delimiter. In addition, we will load the first column as the index of the dataframe as it happens to contain the subject ID's.

In [ ]:
demographics = pd.read_csv(demographics_path, sep="\t", index_col=0)
demographics

We can see some of the standard demographic variables, like "age", "sex", "BMI", and so on. As you might be able to tell, however, this file not *only* contains "demographic" information but also some other participant data, as for example cognitive measurements (e.g. "IST_memory", "IST_fluid").

### Connectomes

Let us now check the connectomes out to see for which subjects we have preprocessed functional connectivity data available.

In [ ]:
connectomes = pd.read_csv(connectomes_path, sep="\t", index_col=0, compression="gzip")
connectomes

In this dataframe again, **each row** corresponds to *one subject* from the study. **Each column** represents a *unique pairwise relationship* between two brain areas (also called an *edge* in graph theory terminology). That is, since a brain parcellation with **N** areas results in an **NxN** symmetric correlation matrix per subject, one half of a subjects matrix is discarded. Similarly, the diagonal of this correlation matrix is also typically discarded as the correlation of an area with itself is always 1. The remaining entries can be stacked and result in one row of this dataframe. Thus, each row contains **N x (N-1) / 2** entries. In our case, since the connectomes were processed with a combination of the Schaefer 100 cortical parcellation and the Tian 32 subcortical parcellation, this results in **100 x (100 - 1) / 2 = 8646** columns. This concept is illustrated by the graphic below:

![title](images/connectomes.png)


### Subsetting the data

As you can see, the first dataframe on demographics contains 928 rows (i.e. subjects), whereas the second dataframe contains 877 rows. Let us for further analyses only select subjects for which we actually have connectomes. But first, let's also make sure, that we identify any 'NaN' values in the functional connectivity data and remove subjects with any 'NaN' entries.

The pandas **isna()** method will check for each entry in the dataframe whether it is 'NaN' or not. That is, if an entry is 'NaN' it will return True and otherwise it will return False. We can use this to identify the indices (i.e. subjects) for which there are 'NaN' entries by combining it with the **any()** method provided by pandas.

First see the output from **isna()**:

In [ ]:
isna = connectomes.isna()
isna

The **any()** method will return whether any element along a given axis (i.e. along a row or a column) is True. The following output should be "True" therefore, if a subject has 'NaN' values and "False" otherwise:

In [ ]:
isna_any = isna.any(axis=1)
isna_any

We can do calculations on these boolean values as if they are 0's and 1's. That is, "True" will be counted as 1 and "False" will be counted as 0. We can therefore use the **.sum()** method to determine the number of 'NaN' values:

In [ ]:
isna_any.sum()

The output ("0") shows us that there aren't any 'NaN' values, so we can simply proceed with the data we have here. Let us therefore now subset the demographic data for which we have connectomes. That is, we will index the demographics dataset using the index from the connectomes dataset:

In [ ]:
subsampled_demographics = demographics.loc[connectomes.index]
subsampled_demographics

The indexing using the **.loc()** method importantly also ensures that the rows in both dataframes are in the same order which will be important later when we convert them to numpy arrays, a data structure that scikit-learn understands.

### Exploring our sample:

Now that the samples in the connectome data and the demographics data are matched, let's take a quick look at sex and age to get an overview of our sample.

The **value_counts()** method takes a pandas series (i.e. a column from the dataframe) and counts the amount of times each possible value is contained in the column. This is a good way of discovering what values are possible for a specific variable, and how many instances there are for each value. This is useful for example when looking at categorial variables, for example "sex":

In [ ]:
subsampled_demographics["sex"].value_counts()

The **plot.hist()** pandas method provides a quick way of making a histogram that we can also group by "sex" to look at each distribution seperately: 

In [ ]:
subsampled_demographics.plot.hist(column=["age"], by="sex", figsize=(10, 8))

As you can see, the age range is quite narrow, and limited to young people. This is a common problem in neuroimaging or psychology studies, which often sample students from their universities for convenience. It is always good to be aware of these limitations before starting any complicated machine learning pipeline.

## Preparing the target

Now, lets try to build a model that can predict a measure of intelligence given a functional connectome. That is, the connectomes will be the features ('X') and the measure of intelligence will be the target ('y'). Let us first take a quick look at our target and its distribution. The **.describe()** method that pandas provides can be used to get a quick look over the relevant descriptive statistics that we may care about:


In [ ]:
subsampled_demographics["IST_intelligence_total"].describe()

We can also quickly plot the histogram and check if there are any NaN values in our target:

In [ ]:
subsampled_demographics.plot.hist(column=["IST_intelligence_total"], by="sex", figsize=(10, 8))

In [ ]:
subsampled_demographics["IST_intelligence_total"].isna().sum()

The results show that the measure of intelligence is roughly normally distributed between values of 100 and 300 with a mean of 200.4 and a standard deviation of 40.77. We can also see that there are two instances with 'NaN' values which we will have to exclude from the analysis. Let's have a look at the 'NaN' samples.

We can use the return value of the **isna()** method to index the original dataframe to see which samples have the 'NaN' values:

In [ ]:
subsampled_demographics[subsampled_demographics["IST_intelligence_total"].isna()]

Likewise, we can use the **~** operator to negate or invert the boolean values of the **isna()** array to index the subjects which do not have 'NaN' values:

In [ ]:
exclude_target_nans = subsampled_demographics[~subsampled_demographics["IST_intelligence_total"].isna()].copy()
exclude_target_nans

The **exclude_target_nans** dataframe now only contains data that actually has data for our target.

We can convince ourselves that this worked correctly by double checking the 'NaN' count. These types of sanity checks (double checking that your code worked even if you think it is obvious can be extremely useful at catching problems early on):

In [ ]:
exclude_target_nans["IST_intelligence_total"].isna().sum()

As expected this returns 0, so we are all good. Now we need to just select the same subjects for the connectomes and make sure that rows are in the same order for both features as well as the target. We can do this by indexing the connectones using the index from our target data:

In [ ]:
subsampled_connectomes = connectomes.loc[exclude_target_nans.index]
subsampled_connectomes

As a last step before we start training models, lets turn the data into a format that sklearn understands: the **numpy array**:

In [ ]:
import numpy as np


X = np.array(subsampled_connectomes)
y = np.array(exclude_target_nans["IST_intelligence_total"])

### Train-test split

Let us first split the data so we have one hold-out validation set that will be left untouched for now.

In [ ]:
from sklearn.model_selection import train_test_split

X_model_selection, X_holdout, y_model_selection, y_holdout = train_test_split(X, y, random_state=25)

### Fitting a bunch of models

Every problem, every classification or regression task is different, and therefore requires a different model. That is, which model works best depends on the underlying processes that generate the distribution of our X and on the "true" function that maps X to y. In other words, we cannot really know which model will work best before we try it out. Let us test out a few popular options therefore starting with a **Ridge Regression**.

In [ ]:
from sklearn.linear_model import Ridge

However, if we check out the **sklearn documentation**, we see that the **Ridge Regression estimator** has a parameter called **alpha**, which can be set to a positive floating point value. This is a **hyperparameter**, *that must be set by the user and cannot be fitted based on the data*. How can we know which value is the best one? First, run the code in the next cell to see the documentation:

In [ ]:
?Ridge

A simple approach would be to try out a number of different **candidate values** for **alpha** and see how well they compare to each other. That is, we could define a **grid** of candidate values, and **search** the candidate that gives us the best outcome. In order to make sure the estimate of the goodness of the outcome (i.e. our estimate of the error) is not dependent on a specific train-test split, we want to perform a cross-validation for every candidate value.

Scikit-learn allows us to perform **model selection using a cross-validated gridsearch** using the **GridSearchCV** object. Let's import it:

In [ ]:
from sklearn.model_selection import GridSearchCV

Let's check out the documentation:

In [ ]:
?GridSearchCV

We can see many parameters, but there are 4 which we care about predominantly:

1. **"estimator"** -> our sklearn estimator object (i.e. the model class)
2. **"param_grid"** -> the grid of hyperparameters to search
3. **"scoring"** -> which scoring metric to use
4. **"cv"** -> the cross-validation scheme to use
   

Let's for the moment, let's go with the example of the ridge regressor, for which we want to tune the alpha value. We can define the estimator as:

In [ ]:
ridge = Ridge()

The **param_grid** parameter typically is handed over as a **dictionary** in which the **keys** consist of the names of the parameters that are to be set for the estimator, and the **values** of the dictionary each yield an **iterable** (for example a **list**) of **possible candidate values** for each parameter.

For example, for our ridge classifier, we can define a very simple grid, that only searches the value for one parameter (the alpha parameter) as follows. Note, that this is only an example grid and should not serve you as a future reference as a grid for a ridge regressor (that is, don't just blindly copy this grid if you want to use a ridge regressor in your work). The values that you choose for your grid can highly depend on your goals and the problem at hand, but it can also help to search the literature that has used the models you want to use and see how they may have defined a grid to search:

In [ ]:
param_grid_ridge = {
    "alpha": [0.001, 0.01, 0.1, 1, 2, 10, 50, 100, 200, 300, 500, 1000]
}

The scoring parameter defines which scoring metric we care about, i.e. which scoring metric should be optimised by the grid search. It can be a string if the metric is already in-built in sklearn. For simplicity we will use ["r2_score"](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html) here, otherwise known as coefficient of determination.

To see what other metrics are available check out this: https://scikit-learn.org/stable/modules/model_evaluation.html

In [ ]:
scoring = "r2"

Lastly, the "cv" parameter can be any scikit-learn compatible cross-validation scheme. Here we will use a simple 5-fold cross-validation. We should also make sure that the KFold cv shuffles the data, but with a specific random state, so that the results are reproducible:

In [ ]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5, shuffle=True, random_state=100)

We can then initialise the GridSearchCV object, and fit it like any other scikit-learn estimator:

In [ ]:
gridsearchcv = GridSearchCV(
    estimator=ridge,
    param_grid=param_grid_ridge,
    scoring=scoring,
    cv=kfold
)

gridsearchcv.fit(X_model_selection, y_model_selection)

The GridSearchCV has an **attribute** called **cv_results_** which we can access as follows:

In [ ]:
gridsearchcv.cv_results_

As you can see it is a dictionary with quite a lot of stuff, and somewhat difficult to read. However, it can be easily converted into a pandas dataframe for easier inspection: 

In [ ]:
cv_results = pd.DataFrame(gridsearchcv.cv_results_)
cv_results

We can see results for each of our 12 model candidates (remember, we used 12 alpha values to define our grid). That is, each row represents the results for 1 model candidate (for which you can see the parameters in the **"params"** column). Perhaps most interesting are the **"mean_test_score"** and **"std_test_score"**, which show us mean accuracy and the standard deviation across the different train-test splits.

In [ ]:
cv_results[["params", "mean_test_score", "std_test_score"]]

We could quickly plot the scoring metric against the alpha parameter to get a quick, more intuitive overview:

In [ ]:
cv_results.plot.line(x="param_alpha", y=["mean_test_score", "std_test_score"])

Although it may be difficult the exact parameter at which **r-squared** is greatest, we can roughly see that a higher alpha increases performance up to an alpha between 200 and 300 and then decreases again (we know from the exact values in the table that alpha=300 gives use the best score, but in a larger, more fine-grained grid this may not be as easy to see).

Conveniently, since the GridSearchCV had the **"refit"** parameter set to True, it already also selects the best scoring model and refits it on all of the data we gave it, so that we can now use it directly for further testing. We can check the best model and its parameters as follows:

In [ ]:
gridsearchcv.best_estimator_

In [ ]:
gridsearchcv.best_params_

Obviously, it's a RidgeClassifier, but we can also see that the alpha value fitted for the best model, corresponds to the alpha parameter for which we can see the highest score in the **cv_results** table. Let us now try to evaluate this best model on the final holdout set:

In [ ]:
holdout_predictions_gscv = gridsearchcv.predict(X_holdout)

In [ ]:
from sklearn.metrics import r2_score

r2_score(y_holdout, holdout_predictions_gscv)

Let us also quickly plot the predicted values versus the actual observed values in a scatter plot to see what our data looks like. Let us for this import seaborns regplot, which lends itself quite nicely to these types of plots:

In [ ]:
import seaborn as sns

predictions_vs_observed = pd.DataFrame(
    {
        "predicted": holdout_predictions_gscv,
        "observed": y_holdout
    }
)
sns.regplot(data=predictions_vs_observed, x="predicted", y="observed")

# Exercises

1. Do a similar grid search for other model families of your choice. Can you search grids with multiple hyperparameters (rather than just the one alpha parameter that we used in the example)?


# Additional Reading or Ressources


The scikit-learn user guide has some good explanations on these topics in its user guide. For example you can look at:

* [computing cross-validated metrics](https://scikit-learn.org/stable/modules/cross_validation.html#computing-cross-validated-metrics)
* [Tuning hyper-parameters using grid search](https://scikit-learn.org/stable/modules/grid_search.html)
* [This article talks a bit more about train-test split evaluation](https://machinelearningmastery.com/train-test-split-for-evaluating-machine-learning-algorithms/)

* The scikit-learn user guide has some explanations and demonstrations of the above mentioned models [here](https://scikit-learn.org/stable/supervised_learning.html) and it usually links to some relevant papers or books as well.
It is well worth trying to read and understand as much as possible about the individual algorithms you are planning to use in your research.

# Sklearn's pipeline

Since we often want to apply preprocessing steps before fitting our models, and we want to do so in a cross-validation consistent way, so that we avoid data leakage and over-optimistic accuracy estimates. We need an easy way to chain different pipeline steps. The scikit-learn **pipeline** object provides just that.

Check out its documentation below:

In [ ]:
from sklearn.pipeline import Pipeline
?Pipeline

The main parameter we care about is the **"steps"** parameter. As the name (and the description) suggests, it is a list of the steps that we want to apply in our pipeline. Let's imagine for example, that we want to make a pipeline in which we first perform a **principal component analysis (PCA)** to extract components, and then fit a **Support Vector Regression (SVR)** on only those extracted components. This may be quite complicated to implement in code especially if we also want to do some hyperparameter tuning in a cross-validated grid search, but combining the two steps in a pipeline makes it much more convenient.

First let's prepare the PCA and the SVR:

In [ ]:
from sklearn.decomposition import PCA
from sklearn.svm import SVR

pca = PCA()
svr = SVR()

Let's also check out the hyperparameters for the SVR:

In [ ]:
?SVR

There are number of different kernels, degrees (for the polynomial kernel), gamma, and C values (you can see already, hyperparameter tuning can get increasingly difficult and complex the more models you start considering). Let us try an SVR with a **"rbf"** kernel and some different values for gamma and C. We also want to set a number of components to extract with the pca. The way we define a grid in a pipeline is to **name the keys of the dictionary** using the **name of the step** and the **name of the parameter** separated by a **double underscore**:

**nameofstep__nameofparameter**

In [ ]:
pca_svr_grid = {
    "pca__n_components": [5, 50],
    "svr__kernel": ["rbf"],
    "svr__C": np.linspace(0.0001, 1000, 5),
    "svr__gamma": np.linspace(0.001, 1000, 5)
}

We also need to define an additional grid to perform the same pipeline without any PCA, as this will be an important setting to try as well:

In [ ]:
no_pca_svr_grid = {
    "pca": ["passthrough"],
    "svr__kernel": ["rbf"],
    "svr__C": np.linspace(0.0001, 1000, 5),
    "svr__gamma": np.linspace(0.001, 1000, 5)
}

Check out the example below and you can see it is really quite simple (Running it will take quite some time):

In [ ]:
steps = [("pca", pca), ("svr", svr)]
pipeline = Pipeline(steps=steps)
gridsearch_pipeline = GridSearchCV(
    estimator=pipeline,
    param_grid=[pca_svr_grid, no_pca_svr_grid],
    cv=KFold(n_splits=5, shuffle=True, random_state=100),
    scoring="r2",
    n_jobs=-1, # for speed up run on all cores
)
gridsearch_pipeline.fit(X_model_selection, y_model_selection)
cv_results = pd.DataFrame(gridsearch_pipeline.cv_results_)
cv_results

This results in quite a big table. We can quickly find the best row using the pandas **.query()** method:

In [ ]:
cv_results.query("rank_test_score == 1")

We can also think about visualising our hyperparameters using some heatmaps (we can make one heatmap for each n_components setting since we can plot the parameters gamma and C on a 2D grid):

In [ ]:
import matplotlib.pyplot as plt

# rename the rows where no pca was performed
cv_results["param_pca__n_components"] = cv_results["param_pca__n_components"].replace(
    {np.nan: "no pca performed"}
)

# prepare a figure
fig, axs = plt.subplots(ncols=1, nrows=4, figsize=(15, 25))

# Loop over every PCA parameter to make one subplot for each PCA setting
for i, pca_n_comps in enumerate(cv_results["param_pca__n_components"].unique()):
    mask = cv_results["param_pca__n_components"] == pca_n_comps
    subsampled_results = cv_results.loc[mask][["param_svr__C", "param_svr__gamma", "mean_test_score"]]
    
    # we do this only because we want to annoying pandas warning about infering
    # numeric data type, we don't care about this being numeric in the plot, as
    # it represents our xtick and ytick labels
    indices = ["param_svr__C", "param_svr__gamma"]
    subsampled_results[indices] = subsampled_results[indices].astype(str)
    
    # this will reshape the data from a "long" to a "wide" format so we can represent
    # it in a heatmap with one parameter on the x axis and one parameter on the y axis
    heatmap_array = subsampled_results.pivot(
        index="param_svr__C", columns="param_svr__gamma", values="mean_test_score"
    ).round(decimals=3)
    
    # this actually makes the subplot
    axs[i].set_title(f"pca components used: {pca_n_comps}")
    sns.heatmap(
        heatmap_array,
        ax=axs[i],
        cmap="coolwarm",
        vmin=cv_results["mean_test_score"].min(),
        vmax=cv_results["mean_test_score"].max(),
        annot=True
    )

Over time you may find that these type of visualisations can be quite helpful when searching over some hyperparameter grids. For example, here we can quickly see that performance is best when not using PCA or when extracting a lot of components rather than only a few.

These kind of visualisations can also help identifz problems with the grid. For example if we find that best performing hyperparameters tend to be towards the boundaries we may want to expand our search grid to include more candidate values. For example in this particular instance we may want to include larger C values to see if we hit a plateau or whether scores will further increase or decrease.

**Importantly**, however, keep in mind that these scores above **are not** final evaluation scores. They merely serve to compare different model candidates, but will be over-optimistic as evaluation scores. **We have to still take the best model and evaluate it on some data that so far the model has not seen.** Let's do this using the GridSearchCV object, which again, has refit the best model on all the training data:

In [ ]:
gridsearch_pipeline.best_estimator_

In [ ]:
holdout_predictions = gridsearch_pipeline.predict(X_holdout)
pipeline_score = r2_score(y_holdout, holdout_predictions)
pipeline_score

Don't be surprised if the score as gone down a bit: This is precisely what is meant when we say that the estimated score during model selection is over-optimistic. If the score drops by a lot, however, this is a good sign that we overfit the model during model selection. Let's quickly plot those predictions again:

In [ ]:
predictions_vs_observed = pd.DataFrame(
    {
        "predicted": holdout_predictions,
        "observed": y_holdout
    }
)
sns.regplot(data=predictions_vs_observed, x="predicted", y="observed")

# Exercise:

1. One very popular method of preprocessing where data leakage can happen quite easily is feature selection. Take one of the feature selection methods [in-built in scikit-learn](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_selection) and add it to a pipeline with a regressor of your choice. Can you do hyperparameter tuning for the feature selection process and the estimator simultaneously?
Hint: check out the scikit-learn user guide to see [how you can use an F-test to select relevant features here](https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection).

# Bonus Material: How to perform a cross-validated grid search for both **model family** and **hyperparameters** simultaneously?

Since we usually want to select the best model for a given problem from a set of **different model families** and their **associated hyperparameters**, you may now wonder how to do this with the GridSearchCV. We can see easily how it is done with one estimator, but it is not easy to see how to do this with a set of different estimators. This is another use case where the scikit-learn pipeline object can come in quite handy!

We can initialise a pipeline with an estimator as a step, and then replace **this estimator** in the **pipeline** with **other type of estimators** using different parameter grids in our **GridSearchCV**, very similar to the way in which we tested the pipeline with and without PCA in the previous example:

In [ ]:
# We only define one step, we care only about the regressor and its hyperparameters.
# We arbitrarily initialise the pipeline with ridge:
pipeline = Pipeline(steps=[("regressor", Ridge())])

# parameters for the ridge regressor
ridge_params = {
    "regressor": [Ridge()], # parameters have to be handed over as iterables!
    "regressor__alpha": np.linspace(0.0001, 1000, 10),
}

# parameters for the support vector regressor:
svr_params = {
    "regressor": [SVR()],
    "regressor__C": np.linspace(0.0001, 1000, 10),
    "regressor__kernel": ["linear", "rbf"],
}

After defining the estimators, the pipeline, and the parameter grids we can put it all together as follows and run the search. This may take a few minutes:

In [ ]:
gridsearch_cv = GridSearchCV(
    estimator=pipeline,
    param_grid=[ridge_params, svr_params],
    scoring="r2",
    cv=kfold,
    n_jobs=-1 # to speed up computation, '-1' means it will use all available CPU's
)
gridsearch_cv.fit(X_model_selection, y_model_selection)
pd.DataFrame(gridsearch_cv.cv_results_)

Let's evaluate again the best model on the holdout data:

In [ ]:
gridsearch_cv.best_estimator_

In [ ]:
gscv_predictions = gridsearch_cv.predict(X_holdout)
r2_score(y_holdout, gscv_predictions)

In [ ]:
predictions_vs_observed = pd.DataFrame(
    {
        "predicted": gscv_predictions,
        "observed": y_holdout
    }
)
sns.regplot(data=predictions_vs_observed, x="predicted", y="observed")